In [53]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
import traceback
from warnings import filterwarnings
filterwarnings('ignore')

In [54]:
driver = None

In [55]:
def open_main_screen():
    global driver
    path = "C:\\Program Files (x86)\\chromedriver.exe"
    service = Service(path)
    options = Options()
    #options.add_argument("--headless")
    chrome_options = Options()
    chrome_options.add_argument("--disable-notifications")  # Bildirimleri devre dışı bırak
    chrome_options.add_argument("--disable-popup-blocking") # Pop-up'ları engelle
    chrome_options.add_argument("--start-maximized")       # Tam ekran başlat
    
    # Tarayıcı profili
    prefs = {"profile.default_content_setting_values.notifications": 2}  # Bildirim izinlerini kapat
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    url = "https://www.trendyol.com/"
    driver.get(url)
    
    try:
        cookies_button_rejected = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-reject-all-handler"]')))
        cookies_button_rejected.click()
    except Exception as e:
        print('cookies not found')
        print(e)


In [56]:
def search_items(entry):
    global driver
    search_box = driver.find_element(By.XPATH,'//*[@id="sfx-discovery-search-suggestions"]/div/div/input')
    search_box.clear()
    search_box.send_keys(entry)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)  

In [57]:
def select_most_reated():
    global driver
    recommended = driver.find_element(By.XPATH,'//*[@id="search-app"]/div/div/div/div[2]/div[1]/div[2]/div/div/div')
    recommended.click()
    most_rated = driver.find_element(By.XPATH,'//*[@id="search-app"]/div/div/div/div[2]/div[1]/div[2]/div/ul/li[7]/span')
    most_rated.click()

In [77]:
def find_dim():
    global driver
    keywords = ["boyut", "ölçü", "uzunluk", "genişlik", "yükseklik", "cm", "mm", "metre",'santimetre','santim']
    pattern = re.compile(r".*(" + "|".join(keywords) + r").*", re.IGNORECASE)
    try:
        find_dem = driver.find_element(By.XPATH,'//*[@id="product-detail-app"]/div/section/div/div/div/ul') 
        relevant_details = [line.strip() for line in find_dem.text.split('\n') if pattern.match(line.strip())]
        texts_with_numbers = [text.strip() for text in relevant_details if re.search(r'\d+\s*[xX]\s*\d+', text)]
        return texts_with_numbers
    except Exception as e:
        return f"An error occurred: {str(e)}" # every product must contain these field so if it's not return me a error.

In [76]:
def find_dim_from_content():
    keywords = ["boyut", "ölçü", "uzunluk", "genişlik", "yükseklik", "cm", "mm", "metre",'santimetre','santim','ebat']
    pattern = re.compile(r".*(" + "|".join(keywords) + r").*", re.IGNORECASE)
    try:
        find_dem = driver.find_element(By.XPATH, '//*[@id="urun-aciklamasi"]/div')
        relevant_details = [line.strip() for line in find_dem.text.split('\n') if pattern.match(line.strip())]
        texts_with_numbers = [text.strip() for text in relevant_details if re.search(r'\d+\s*[xX]\s*\d+', text)]
        return texts_with_numbers
    except Exception as e:
        return None #every product don't have to contain these field so if it's not, note return me any thing

In [78]:
def guess_dimensions():
    global driver
    try:
        element = driver.find_element(By.CLASS_NAME, 'navigate-all-reviews-btn')
    except:
        return None
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", element)
    time.sleep(1)
    #driver.execute_script("window.scrollBy(0, -400);") bu biraz daha kesin olmayan yol
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[p[text()='boyut ve ölçü']]"))).click()
    except:
        try:
            button = driver.find_element(By.XPATH, "//button[contains(@class, 'tags-wrapper__button') and not(contains(@class, 'rotate-180'))]")
            button.click()
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[p[text()='boyut ve ölçü']]"))).click()
        except:
            return None

    time.sleep(1)
    answers = driver.find_elements(By.CLASS_NAME, 'answer')
    for answer in answers:
        answer = answer.text.split('\n')[1]
        answer = answer.lower()
        if re.search(r'\d+\s*[xX]\s*\d+', answer):
            return answer

In [79]:
def guess_dim_from_comment(entry = 'Boyut' ):
    try:
        element = driver.find_element(By.CLASS_NAME, 'navigate-all-questions-btn')
    except:
        return None
    element.click()
    time.sleep(2)
    search_box = driver.find_element(By.XPATH,'//*[@id="questions-and-answers-app"]/div/div/div/div/div/div[3]/div[2]/div/div[1]/input')
    search_box.send_keys(entry)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2)

    answers = driver.find_elements(By.CLASS_NAME, 'answer')
    for answer in answers:
        answer = answer.text.split('\n')[1]
        answer = answer.lower()
        if re.search(r'\d+\s*[xX]\s*\d+', answer):
            return answer
    return None
    

In [63]:
def click_all_features():
    global driver
    ozellikler = driver.find_element(By.CLASS_NAME,'button-all-features')
    ozellikler.click()

In [64]:
def close_understood():
    global is_it_first_time
    if is_it_first_time:
        global driver
        understood = driver.find_element(By.XPATH,'//*[@id="product-detail-app"]/div/div[2]/div/div[2]/div[2]/div/div[1]/aside/div[1]/div/div[2]/div/div[2]/div/div/button')
        understood.click()
        is_it_first_time = False

In [65]:
def scrap_brand():
    global driver
    brand = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//*[contains(@class, 'product-brand-name')]")))
    return brand.text

In [66]:
def scrap_title():
    all_title = driver.find_element(By.CLASS_NAME,'pr-new-br').text
    brand = scrap_brand()
    title  = all_title.strip(brand).strip()
    return brand, title

In [62]:
def scrape_photo():
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "rvw-cnt-tx"))).click()
    #enter_images = driver.find_element(By.XPATH, "//span[@class='ps-tags__tag--bold' and contains(text(), 'fotoğraflı')]")
    enter_images = driver.find_element(By.XPATH,'//*[@id="rating-and-review-app"]/div/div/div/div[3]/div/div/div[3]/div[1]/div/div[1]/button')
    enter_images.click()

In [92]:
def run():
    global driver
    global is_it_first_time
    is_it_first_time = True
    open_main_screen()
    search_keys = ['suluk']
    for key in search_keys:
        search_items(key)
        select_most_reated()
        time.sleep(2)
        products = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'variant-options-wrapper')))
        for i in products:
            i.click()
            time.sleep(2)
            windows = driver.window_handles
            driver.switch_to.window(windows[1])
            if is_it_first_time:
                close_understood()
            #dfdfdfdfdfffddfdfdfdfdfdfdfdfffddfdfdf
            time.sleep(2)
            brand = scrap_title()[0]
            title = scrap_title()[1]
            key = key
            #dfdfdfdfdfffddfdfdfdfdfdfdfdfffddfdfdf
            click_all_features()
            time.sleep(1)
            dim = find_dim()
            if not dim:
                driver.execute_script("window.scrollBy(0, 400);") #bazi urunlerde aciklama olmadigi icin ortala yerine kaydiriyorum.
                dim = find_dim_from_content()
                if not dim:
                    dim = guess_dimensions()
                    if not dim:
                        driver.back()
                        dim = guess_dim_from_comment()
            driver.close()
            driver.switch_to.window(windows[0])
            driver.execute_script("window.scrollTo(0, 0);")
            print(dim)
            break

In [93]:
run()

None


In [ ]:
windows = driver.window_handles
driver.switch_to.window(windows[1])
scrap_brand()

In [83]:
windows = driver.window_handles
driver.switch_to.window(windows[1])
driver.execute_script("window.scrollBy(0, 400);")